## Professional Athlete Classifier

#### Connect To Workspace

In [6]:
from azureml.core import Workspace

ws = Workspace.from_config()

#### Data Prep

Use Azure Cognitive Services to search internet for photos of NBA players and then upload to blob storage

- TODO: Add instructions on how to set up cognitive services
- TODO: Add packages o install (cognitive service sdk)

In [8]:
import os
import sys

data_dir = 'data'
!{sys.executable} cognitive-service-web-scraper.py --root_dir {data_dir}

Writing image 0 for Stephen Curry...
Writing image 1 for Stephen Curry...
Writing image 2 for Stephen Curry...
Writing image 3 for Stephen Curry...
Writing image 4 for Stephen Curry...
Writing image 5 for Stephen Curry...
Writing image 6 for Stephen Curry...
Writing image 7 for Stephen Curry...
Writing image 8 for Stephen Curry...
Writing image 9 for Stephen Curry...
Writing image 10 for Stephen Curry...
Writing image 11 for Stephen Curry...
Writing image 12 for Stephen Curry...
Writing image 13 for Stephen Curry...
Writing image 14 for Stephen Curry...
Writing image 15 for Stephen Curry...
Writing image 16 for Stephen Curry...
Writing image 17 for Stephen Curry...
Writing image 18 for Stephen Curry...
Writing image 19 for Stephen Curry...
Writing image 20 for Stephen Curry...
Writing image 21 for Stephen Curry...
Writing image 22 for Stephen Curry...
Writing image 23 for Stephen Curry...
Writing image 24 for Stephen Curry...
Writing image 25 for Stephen Curry...
Writing image 26 for S

Writing image 17 for Kawhi Leonard...
Writing image 18 for Kawhi Leonard...
Writing image 19 for Kawhi Leonard...
Writing image 20 for Kawhi Leonard...
Writing image 21 for Kawhi Leonard...
Writing image 22 for Kawhi Leonard...
Writing image 23 for Kawhi Leonard...
Writing image 24 for Kawhi Leonard...
Writing image 25 for Kawhi Leonard...
Writing image 26 for Kawhi Leonard...
Writing image 27 for Kawhi Leonard...
Writing image 28 for Kawhi Leonard...
Writing image 29 for Kawhi Leonard...
Writing image 30 for Kawhi Leonard...
Writing image 31 for Kawhi Leonard...
Writing image 32 for Kawhi Leonard...
Writing image 33 for Kawhi Leonard...
Writing image 34 for Kawhi Leonard...
Writing image 35 for Kawhi Leonard...
Writing image 36 for Kawhi Leonard...
Writing image 37 for Kawhi Leonard...
Writing image 38 for Kawhi Leonard...
Writing image 39 for Kawhi Leonard...
Writing image 40 for Kawhi Leonard...
Writing image 41 for Kawhi Leonard...
Writing image 42 for Kawhi Leonard...
Writing imag

Writing image 35 for James Harden...
Writing image 36 for James Harden...
Writing image 37 for James Harden...
Writing image 38 for James Harden...
Writing image 39 for James Harden...
Writing image 40 for James Harden...
Writing image 41 for James Harden...
Writing image 42 for James Harden...
Writing image 43 for James Harden...
Writing image 44 for James Harden...
Writing image 45 for James Harden...
Writing image 46 for James Harden...
Writing image 47 for James Harden...
Writing image 48 for James Harden...
Writing image 49 for James Harden...
Writing image 50 for James Harden...
Writing image 51 for James Harden...
Writing image 52 for James Harden...
Writing image 53 for James Harden...
Writing image 54 for James Harden...
Writing image 55 for James Harden...
Writing image 56 for James Harden...
Writing image 57 for James Harden...
Writing image 58 for James Harden...
Writing image 59 for James Harden...
Writing image 60 for James Harden...
Writing image 61 for James Harden...
W

In [9]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir=data_dir, target_path='athletes')

Uploading an estimated of 505 files
Target already exists. Skipping upload for athletes/James_Harden/0.jpg
Target already exists. Skipping upload for athletes/James_Harden/1.jpg
Target already exists. Skipping upload for athletes/James_Harden/10.jpg
Uploading data/James_Harden/100.jpg
Target already exists. Skipping upload for athletes/James_Harden/11.jpg
Target already exists. Skipping upload for athletes/James_Harden/12.jpg
Target already exists. Skipping upload for athletes/James_Harden/13.jpg
Target already exists. Skipping upload for athletes/James_Harden/14.jpg
Target already exists. Skipping upload for athletes/James_Harden/15.jpg
Target already exists. Skipping upload for athletes/James_Harden/16.jpg
Target already exists. Skipping upload for athletes/James_Harden/17.jpg
Target already exists. Skipping upload for athletes/James_Harden/18.jpg
Target already exists. Skipping upload for athletes/James_Harden/19.jpg
Target already exists. Skipping upload for athletes/James_Harden/2

Target already exists. Skipping upload for athletes/Kawhi_Leonard/14.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/15.jpg
Uploaded data/James_Harden/99.jpg, 53 files out of an estimated total of 472
Target already exists. Skipping upload for athletes/Kawhi_Leonard/16.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/17.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/18.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/19.jpg
Uploaded data/James_Harden/88.jpg, 54 files out of an estimated total of 473
Target already exists. Skipping upload for athletes/Kawhi_Leonard/2.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/20.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/21.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/22.jpg
Target already exists. Skipping upload for athletes/Kawhi_Leonard/23.jpg
Target already exists. Skipping upload for a

Target already exists. Skipping upload for athletes/Kevin_Durant/33.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/34.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/35.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/36.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/37.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/38.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/39.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/4.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/40.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/41.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/42.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/43.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/44.jpg
Target already exists. Skipping upload for athletes/Kevin_Durant/

Target already exists. Skipping upload for athletes/Lebron_James/64.jpg
Target already exists. Skipping upload for athletes/Lebron_James/65.jpg
Target already exists. Skipping upload for athletes/Lebron_James/66.jpg
Target already exists. Skipping upload for athletes/Lebron_James/67.jpg
Target already exists. Skipping upload for athletes/Lebron_James/68.jpg
Target already exists. Skipping upload for athletes/Lebron_James/69.jpg
Target already exists. Skipping upload for athletes/Lebron_James/7.jpg
Target already exists. Skipping upload for athletes/Lebron_James/70.jpg
Target already exists. Skipping upload for athletes/Lebron_James/71.jpg
Target already exists. Skipping upload for athletes/Lebron_James/72.jpg
Target already exists. Skipping upload for athletes/Lebron_James/73.jpg
Target already exists. Skipping upload for athletes/Lebron_James/74.jpg
Target already exists. Skipping upload for athletes/Lebron_James/75.jpg
Target already exists. Skipping upload for athletes/Lebron_James/

Target already exists. Skipping upload for athletes/Stephen_Curry/97.jpg
Target already exists. Skipping upload for athletes/Stephen_Curry/98.jpg
Target already exists. Skipping upload for athletes/Stephen_Curry/99.jpg
Uploaded 68 files


$AZUREML_DATAREFERENCE_1ca29405bf0643efae03140e1c0d5f69

## Train

- Requirement packages to add: imutils, opencv-python